In [1]:
import pandas as pd
df = pd.read_csv('/Users/202837/Documents/Python/Food Price API/RecipeNLG_dataset.csv', index_col=[0])

In [5]:
import requests
import json
import re
from bs4 import BeautifulSoup

In [3]:
df.columns

Index(['title', 'ingredients', 'directions', 'link', 'source', 'NER'], dtype='object')

In [6]:
def getPrice(ing):
    ingList = ing.strip('][').replace('\"', '').replace(", ", "\\n")

    widget = requests.post(
        "https://api.spoonacular.com/recipes/visualizePriceEstimator",
        data={
            "apiKey": "17a880d8a40f4451bccdc80f4d3aa38e",
            "ingredientList": ingList,
            "servings": "1",
            "defaultCss": "false",
        },
    )
    soup = BeautifulSoup(widget.text, "html.parser")
    things = soup.select_one("#spoonacularPriceBreakdownTable").find_all("div")

    items = repr(things[2]).split("<br/>")[1:-1]
    prices = repr(things[3]).split("<br/>")[1:-1]

    ingredients = dict(zip(items, prices))
    return ingredients


In [98]:
amount = 50
prices = df.ingredients.iloc[:amount].apply(getPrice) 

In [99]:
print(prices.iloc[49])

{'Â½ pounds ground beef': '$1.76', '1 package dry a-soup onion soup mix': '$0.01', '1 pound canned tomatoes': '$0.97', '8 ounces canned tomato sauce': '$0.89', '10 ounces frozen mixed vegetables': '$1.11'}


In [4]:
prices.name = "Price"
# print(prices)
print(df.head())
outputDf = df.iloc[:50].assign(ingBreak = prices.values)

NameError: name 'prices' is not defined

In [105]:
pd.DataFrame(prices.values[0].keys(), prices.values[0].values)

array([{'1 cup firmly brown sugar': '$0.71', 'Â½ cups evaporated milk': '$0.30', 'Â½ teaspoons vanilla': '$0.16', 'Â½ cups broken nuts': '$0.87', '2 Tbsps butter': '$0.16', '3.5 cups shredded bite size rice biscuits': '$2.08'},
       {'some cut up': '$0.36', '4 boned chicken breasts': '$8.02', '1 can canned cream of mushroom soup': '$1.31'},
       {'32 ounces frozen corn': '$4.21', '8 ounces cream cheese': '$2.35', 'â\x85\x93 cups butter': '$0.65', 'Â½ teaspoons garlic powder': '$0.05', 'Â¼ teaspoons pepper': '$0.01'},
       {'1 large whole chicken': '$5.06', '21 ounces canned chicken gravy': '$7.89', '10.5 ounces canned cream of mushroom soup': '$1.28', '6 ounces stove top stuffing': '$1.52', '4 ounces shredded cheese': '$1.21'},
       {'1 cup peanut butter': '$0.92', 'Â¾ cups graham cracker crumbs': '$0.70', '1 cup butter': '$1.95', '3.5 cups powdered sugar': '$1.35'},
       {'6 baking potatoes': '$1.70', '1 pound lean extra ground beef': '$5.13', 'â\x85\x94 cups butter': '$0.86

In [134]:
outputDf.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER,ingBreak
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...","{'1 cup firmly brown sugar': '$0.71', 'Â½ cups..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom...","{'some cut up': '$0.36', '4 boned chicken brea..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar...","{'32 ounces frozen corn': '$4.21', '8 ounces c..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo...","{'1 large whole chicken': '$5.06', '21 ounces ..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu...","{'1 cup peanut butter': '$0.92', 'Â¾ cups grah..."
